In [1]:
from bs4 import BeautifulSoup
import urllib3
import re
import json
import pandas as pd
import time

In [2]:
url = "https://www.johnsonwilshire.com/"
def get_html(url):
    http = urllib3.PoolManager()
    response = http.request('GET', url,headers = {'User-agent': 'your bot 0.1'})
    soup = BeautifulSoup(response.data)
    return soup,response.status

In [3]:
links = []
soup,status = get_html(url)
for a_tag in soup.find_all('a', {'data-testid': 'linkElement'}):
    href = a_tag.get('href')
    if href:
        links.append(href)

In [4]:
all_links = []
for link in links:
    x = link.split('https://www.johnsonwilshire.com')
    if x[0]=='' and x[1]!='':
        all_links.append(link)

In [5]:
# all_links

In [6]:
from tqdm.notebook import tqdm
all_products = []
for link in tqdm(all_links):
    soup,status = get_html(link)
    for div_tag in soup.find_all('div', {'data-hook': 'product-item-root'}):
        data_slug = div_tag.get('data-slug')
        all_products.append('https://www.johnsonwilshire.com/product-page/'+data_slug)

  0%|          | 0/53 [00:00<?, ?it/s]

In [7]:
def get_details(url):
    soup,status = get_html(url)
    
    if status!=200:
        return [url] + [status]*4
    
    script_tag = soup.find('script', type='application/ld+json')
    json_data = json.loads(script_tag.string)
    image_urls = [image["contentUrl"] for image in json_data["image"] if image.get("representativeOfPage")]
    
    image_link = '\n'.join(image_urls)

    # Extract the title
    title_tag = soup.find('meta', {'name': 'twitter:title'})
    title = title_tag['content'] if title_tag else ''

    description_tag = soup.find('meta', {'name': 'twitter:description'})
    description = description_tag['content'] if description_tag else ''
    if description:
        description = description.strip().replace('\t',',')
    
    description = re.sub(r',,+', ',', description)
    
    info_section = soup.find_all('div', {'data-hook': 'info-section-description'})
    titles = soup.find_all('h2', {'data-hook': 'info-section-title'})
 
    desc = ''
    for t,info in zip(titles,info_section):
        desc += t.get_text(strip=True) + ': ' + info.get_text(separator=",",strip=True) + '\n'
    
    desc = re.sub(r',,+', ',', desc)
        
    return url,image_link,title,description,desc

In [8]:
df = pd.DataFrame([], columns=['url','image', 'title', 'Desc', 'Long Desc'])

In [9]:
for item in tqdm(all_products):
    try:
        url,image_link,title,description,desc = get_details(item)
    except:
        print(url)
    df.loc[len(df)] = [url,image_link,title,description,desc]
    time.sleep(1)

  0%|          | 0/242 [00:00<?, ?it/s]

https://www.johnsonwilshire.com/product-page/green-heavy-duty-tarps
https://www.johnsonwilshire.com/product-page/kevlar-sewn-6010k


In [14]:
df.drop_duplicates(inplace=True)

In [11]:
# dfa = df.copy()

In [12]:
df['title'] = df['title'].apply(lambda x: x.split('|')[0].strip() if x else None)

In [15]:
df.to_csv('all_items1.csv',index=False)

In [23]:
# clean image url

import pandas as pd
df = pd.read_csv('all_items1.csv')

In [24]:
f = lambda x: x.split('.jpg')[0]+'.jpg' if '.jpg' in x else x.split('.png')[0]+'.png'

df['image'] = df['image'].apply(f)

In [25]:
df.to_csv('all_items2.csv',index=False)